In [ ]:
#신영

In [1]:
import numpy as np
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import matplotlib.pyplot as plt
import os
import librosa
import soundfile as sf # 리샘플링한 음악을 저장
import pandas as pd
import csv
# from tqdm import tqdm 

In [2]:
print()

In [11]:
# 사용한 음향데이터가 16000hz가 아니라 경고가 나므로 음향의 샘플링을 16000hz로 변환

def resampling(file,origin_sr,resample_sr):
#     resample_sr = 16000
#     origin_sr = 44100
#     resample_sr= resample_sr
#     origin_sr = origin_sr
    
    # 리샘플링을 해줘야하니 input으로 넣어줄 sound파일이 있는 경로 지정
    os.chdir("C:/Users/husky/Documents/기업프로젝트/data/freesound-audio-tagging/audio_test")

    # file 예시: 00063640.wav ,  (type: 'str')
    y, sr = librosa.load(file, sr=origin_sr)
    resample = librosa.resample(y, sr, resample_sr)
#     print("original wav sr: {}, original wav shape: {}, resample wav sr: {}, resmaple shape: {}".format(origin_sr, y.shape, resample_sr, resample.shape))
    # 리샘플링이 된 wav 로 파일 저장 경로 설정
    sf.write('../test_music/' + file, resample, resample_sr, format='WAV', endian='LITTLE', subtype='PCM_16')

#모든 파일을 리샘플링(44100hz->16000hz) 해주었다면 더이상 실행시키지 않아도 됨
# 추후 새로 입력이 들어오는 데이터가 있다면 해당 파일의 샘플링을 알아야하고,
# 16000이 아니라면 샘플링을 해주어야 함(샘플링이 다르면 경고가 발생하는데 근대 무시해도 문제는 없다고 ..)

In [12]:
#  ElasticSearch는 저장해야할 picture가 늘어나면 강제종료되는 버그가 있기에 파일에 저장하는 방식을 선택

def feature_save(file):
     # 리샘플링이 된 wav 파일의 경로를 가져옴
    os.chdir("../test_music/")
    
    # wav파일에서 MFCC 특성 추출
    (rate,sig) = wav.read(file)
    
    mfcc_feat = mfcc(sig,rate)
    feat_mean = np.mean(mfcc_feat, axis = 0)
    feat_cov = np.cov(mfcc_feat,rowvar=0)
    feat_cov_up = np.triu(feat_cov, k=0)# 상삼각행렬(Upper triangular matrix)을 반환
    feat_flat = feat_cov_up.flatten()
    index = np.arange(len(feat_flat))[feat_flat==0]
    feat_cov = list(np.delete(feat_flat, index))
    
    # 모든 특성 벡터를 더한 값 
    feat = []
    sum1= np.sum(feat_mean)+sum(feat_cov)
    sum2=np.float64(sum1).item() # 리스트에 넣기 위해 np float64 -> list형식으로 변환
    
#       생성되는 csv 형태
#             name         sum         vector1     vector2     vector3  ... vector104
#       1번째 wav파일   벡터 합       벡터 값1    벡터 값2   벡터 값3 ...   벡터 값104
#       2번재 wav파일   벡터 합        ... 
    feat.append(file) # wav파일 이름 저장
    feat.append(sum2) # extend는 list에 여러 이터레이터를 인풋, append는 한개의 항목 인풋
    feat.extend(feat_mean)#<class 'numpy.ndarray'>   # 이 값은 들어가짐. extend()라 그런듯
    feat.extend(feat_cov) # <class 'list'>
    
    # 'test.csv' 파일은 9000개 가량의 음향을 MFCC 특성추출해서 저장한 csv파일임
    # 생성되어있는 csv파일의 마지막에 MFCC로 특성추출해준 새로운 음향 벡터를 기록
    with open('../test.csv', 'a', newline='\n') as f:
        writer = csv.writer(f)
        writer.writerow(feat)

In [63]:
def music_search(file):
    # csv 읽기
    music_df=pd.read_csv('./test.csv',encoding="utf-8")

    # 마지막에 추가된 값(비교할 값)의 sum을 compare_sum에 저장
    compare_sum=music_df.iloc[-1]['sum']
    # 유사도 비교해줄 행을 compare에 저장
    compare=music_df.iloc[-1]

    # 1차 필터링
    # sum값이 인접한 10%들을 검색하여 filter1에 저장
    filter1=music_df[(music_df['sum']>(compare_sum*0.95)) & (music_df['sum']<(compare_sum*1.05))]
    # 비교해야할 값도 filter1에 포함되었기에 filter1에선 삭제
    filter1=filter1.drop(filter1.index[-1])
    
    
    # 2차 필터링
    hap=0
    filter_list=[]
    for i in range(len(filter1)):
        for j in range(2,len(filter1.columns)): # 2부터 시작
            hap+= (compare[j]-filter1.iloc[i][j])**2 # 유클리드 거리 
        filter_list.append(hap)
        hap=0

    # 정렬을 위해 dict로 생성 
    filter_dict={}
    for i in range(len(filter_list)):
        filter_dict[filter1.iloc[i]['name']]=filter_list[i]

    # 값(value)을 기준으로 오름차순 정렬
    sorted(filter_dict.items(), key=lambda x:x[1])
    
    # 3차 필터링
    # 해당 딕셔너리에 서로 똑같은 value값이 있다면 제외( 똑같은 음향파일을 중복해서 추천해주는 경우 제외 )
    dict_len=len(filter_dict)
    list_del=[]
    
    for idx, (key,element) in enumerate(filter_dict.items()):
        for sub_idx in range(idx+1, dict_len):
            if(element == list(filter_dict.values())[sub_idx]):
                list_del.append(list(filter_dict.keys())[sub_idx])
                
    if(list_del):# 비어있지않다면 실행
        for d in list_del:
            del filter_dict[d]
            
        
    # 유사한 음향 5개 반환
    result=[]
    cnt=0
    for key in filter_dict.keys():
        if(cnt>=5):
            break
        # file이름이 똑같다면 넘김
        if(file==filter_dict.keys):
            continue
        result.append(key)
        cnt+=1
#     print(result)
    return result

In [64]:
# test code


try:
# 유사음향을 찾을 file이름을 줘야함
    file='00ae03f6.wav'
    resampling(file,44100,16000) # 44100은 input 샘플링, 16000은 바꿀 샘플링
    feature_save(file)
    music_list=music_search(file)
    print(music_list)
except:
    print(file,'error')

['002d256b.wav', '06730f97.wav', '0b142b38.wav', '0d649d1b.wav', '0f18c55b.wav']


In [65]:
# input음향 -> 샘플링 조절(안해도 되긴 함, 경고가 거슬릴뿐..) -> 특성벡터 추출 -> 한 행의 특성벡터를 전부 더한 sum값 생성 
# -> (파일 이름, sum값, 특성 벡터들(1~106))형태로 csv파일에 저장 : 완료

# 1차 분류
# (전체 데이터에서 10%정도로 1차 필터링)
# 음향 특성추출벡터에 sum값을 비교하여 근접하는 sum값을 갖는 벡터로 분류

# 2차 분류
# 벡터간의 거리 계산하여 유사도 분류
### 시도해볼 방법 ( knn, 코사인유사도, svm, 유클리드 제곱거리, 랜덤포레스트)
# knn은 이미지를 분류하기에 적합하지않다고 함
# 코사인유사도는 해보니 벡터간의 거리 계산하는데 음향 추출벡터라 값을 정규화시키기엔 애매한 부분이 있었음
# 유클리드 거리는 시도안해봤는데 거리 계산해서 나온 값이 작아야 유사한지 커야 유사한지 잘 모르겠음 : 최소가 되어야 함
#  -> 유클리드 거리로 계산하였다 ( 유클리드 제곱거리도 있던데 어떤게 좋은지는 연구필요합니다 )
# 클러스터링을 만들어서 세부적으로 분석 -> k-means

# 3차 분류 
# 지금은 문제없는데 추후에 똑같은 음향이 많이 들어간다면 중복된 음향이 여러개가 같이 추천될 수 있음
# MFCC 음향 벡터 값이 서로 같다면 이는 같은 음악이니 추천 리스트에서 중복된 음향을 삭제해준다.